In [148]:
import pandas as pd
import numpy as np

excel_reader = pd.ExcelFile('./data.xlsx')  
sheet_names = excel_reader.sheet_names  # 读取文件的所有表单名，得到列表
df_sheet =  [excel_reader.parse(sheet_name=sheet_names[i]) for i in range(3)]

df_sheet[0] = df_sheet[0].rename(columns={
    "文物编号" : "id",
    "纹饰"     : "ornament",
    "类型"     : "type",
    "颜色"     : "color",
    "表面风化" : "degree",
})

df_sheet[1] = df_sheet[1].rename(columns={
    "文物采样点"       : "id",
    "二氧化硅(SiO2)"   : "sio",
    "氧化钠(Na2O)"     : "nao",
    "氧化钾(K2O)"      : "ko",
    "氧化钙(CaO)"      : "cao", 
    "氧化镁(MgO)"      : "mgo",
    "氧化铝(Al2O3)"    : "alo",
    "氧化铁(Fe2O3)"    : "feo",
    "氧化铜(CuO)"      : "cuo",
    "氧化铅(PbO)"      : "pbo",
    "氧化钡(BaO)"      : "bao",
    "五氧化二磷(P2O5)" : "po",
    "氧化锶(SrO)"      : "sro",
    "氧化锡(SnO2)"     : "sno",
    "二氧化硫(SO2)"    : "so",
})

df_sheet[2] = df_sheet[2].rename(columns={
    "文物编号"         : "id",
    "表面风化"         : "degree",
    "二氧化硅(SiO2)"   : "sio",
    "氧化钠(Na2O)"     : "nao",
    "氧化钾(K2O)"      : "ko",
    "氧化钙(CaO)"      : "cao", 
    "氧化镁(MgO)"      : "mgo",
    "氧化铝(Al2O3)"    : "alo",
    "氧化铁(Fe2O3)"    : "feo",
    "氧化铜(CuO)"      : "cuo",
    "氧化铅(PbO)"      : "pbo",
    "氧化钡(BaO)"      : "bao",
    "五氧化二磷(P2O5)" : "po",
    "氧化锶(SrO)"      : "sro",
    "氧化锡(SnO2)"     : "sno",
    "二氧化硫(SO2)"    : "so",
})

for i in range(3):
    df_sheet[i].to_csv(f"sheet{i}_origin.csv",index=False)

In [149]:
from sklearn.preprocessing import LabelEncoder 

def to_num(df: pd.DataFrame, col):
    le = LabelEncoder()
    types = df[col].drop_duplicates().sort_values()
    df[col] = le.fit_transform(df[col])
    for type_ in types:
        if type(type_) == float and np.isnan(type_):
            print("Nan _")
        else:
            print(type_, le.transform((type_,))[0])


In [150]:
to_num(df_sheet[0], 'ornament')


A 0
B 1
C 2


In [151]:
to_num(df_sheet[0], 'type')


铅钡 0
高钾 1


In [152]:
to_num(df_sheet[0], 'color')


浅绿 0
浅蓝 1
深绿 2
深蓝 3
紫 4
绿 5
蓝绿 6
黑 7
Nan _


In [153]:
to_num(df_sheet[0], 'degree')


无风化 0
风化 1


In [124]:
df_sheet[0].to_csv("sheet0_num.csv", index=False)


In [128]:
translater = {
    "铅钡" : "lead_barium",
    "高钾" : "high_potassium",
    "A" : "A",
    "B" : "B",
    "C" : "C",
    "蓝绿" : "blue_green",
    "浅绿" : "light_green",
    "深绿" : "dark_green",
    "深蓝" : "deep_blue",
    "浅蓝" : "light_blue",
    "紫"   : "purple",
    "绿"   : "green",
    "黑"   : "black",
    "无风化" : "none_weathered",
    "风化"   : "weathered",
    "0" : "none_weathered",
    "1" : "weathered"
}

def to_one_hot(df: pd.DataFrame, col):
    df_one_hot = pd.get_dummies(df[col])
    df_one_hot.columns = df_one_hot.columns.map(translater)
    return pd.concat([df, df_one_hot], axis=1, join='inner').drop(columns=[col])

ret = to_one_hot(pd.read_csv('sheet0_origin.csv'), 'ornament')
ret = to_one_hot(ret, 'type')
ret = to_one_hot(ret, 'color')
ret = to_one_hot(ret, 'degree')
ret.to_csv("sheet0_one_hot.csv", index=False)


In [14]:
import pandas as pd
import numpy as np

# 归一化

sheet1 = pd.read_csv('./sheet1_1.csv').fillna(0)
sheet1 = pd.concat([sheet1[['id','weathered']], sheet1.drop(columns=['id','weathered']).apply(lambda x: (x-np.mean(x)/np.std(x)))], axis=1)

sheet1.to_csv("sheet1_2.csv",index=False)

In [2]:
import pandas as pd

sheet0 = pd.read_csv("./sheet0_num.csv")
sheet1 = pd.read_csv("./sheet1_2.csv")

type_match = {
    0 : '铅钡',
    1 : '高钾',
}

sheet0 = sheet0[['id', 'type']]

data = sheet1.merge(sheet0, left_on='id', right_on='id', how='inner')

data.to_csv("sheet1_3.csv",index=False)

for type_ in range(2):
    data1 = data[data.type == type_]
    data1 = data1.drop(columns=['type'])
    data1.to_csv(f"sheet1_{type_match[type_]}.csv", index=False)


In [38]:
import pandas as pd

all_columns = ['sio', 'nao', 'ko', 'cao', 'mgo', 'alo', 'feo', 'cuo', 'pbo', 'bao', 'po', 'sro', 'sno', 'so']

data = pd.read_csv("./sheet1_1.csv").fillna(0)

DROP_RATE = 0.25

def rip_off_mass_nan_columns(df, columns, drop_rate):
    chosen_columns = []
    dropted_columns = []
    TOT_NUM = len(df)
    for col_name in columns:
        if len(data[data[col_name] < 1e-2]) / TOT_NUM < drop_rate:
            dropted_columns.append(col_name)
        else:
            chosen_columns.append(col_name)
    return chosen_columns, dropted_columns

print(rip_off_mass_nan_columns(data, all_columns, DROP_RATE))

(['nao', 'ko', 'mgo', 'feo', 'bao', 'sro', 'sno', 'so'], ['sio', 'cao', 'alo', 'cuo', 'pbo', 'po'])
